### Get data from gaia

In [3]:
#imports
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [9]:
#gaia query
job = Gaia.launch_job_async("SELECT ALL gaia_source.source_id,gaia_source.ra,gaia_source.ra_error,gaia_source.dec,gaia_source.dec_error,gaia_source.pmra,gaia_source.pmra_error,gaia_source.pmdec,gaia_source.pmdec_error,gaia_source.phot_bp_mean_mag,gaia_source.radial_velocity,gaia_source.radial_velocity_error FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',183.721,12.785,3))=1  AND  (gaiadr2.gaia_source.phot_bp_mean_mag<=17)")

Query finished.


In [10]:
#resulting datatable
r = job.get_results()
print(r['ra','dec'])

        ra                dec        
       deg                deg        
------------------ ------------------
182.06770195328755  11.79359280719235
181.25441451210122 11.008729614018321
181.53595754136217 11.184619989802899
181.99360808226916 11.587464938436003
181.48408156403372  11.31517732538975
182.04456959710964 11.780362569798461
181.59843013913613 11.175090701328484
182.54767299559865 12.083720544056007
 181.5548019301534 11.873944575898046
  182.013979921151 11.817131925823952
               ...                ...
 185.1716258720525 15.143946296280525
 185.2083723033049  15.27356951100647
185.23546139985038  15.30806853903222
185.30742776507591  15.31761270860593
185.10192943238832 15.143515510864267
185.29110244593113 15.361113183055878
184.70607105903395  15.43585725415703
 184.9304420448486 15.204390789719593
184.96258573048792 15.394453952181477
184.70304909052405  15.48668823506751
Length = 9514 rows


### Read csv (apt) file

In [20]:
#imports
import pandas as pd 

In [21]:
#read 
data = pd.read_csv("ca2183_apt.csv") 

In [22]:
data

,Number,ApertureRA,ApertureDec,CentroidRA,CentroidDec,ApertureX,ApertureY,CentroidX,CentroidY,CentroidFlag,...,ApertureMinorRadius,ApertureRotationAngle,SkyRadiusInner,SkyRadiusOuter,ApertureNumPixels,ApertureNumRejected,SkyNumPixels,RadialProfileFWHM,Flags,Image
0,1,185.754367,13.825964,185.753632,13.826088,991,13,990.870,13.712,1,...,5,0,8,15,79.04,0.0,500,3.31,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
1,2,185.752261,13.696440,185.751446,13.696436,1120,16,1119.996,16.791,1,...,5,0,8,15,79.04,0.0,513,4.05,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
2,3,185.755977,13.614154,185.755287,13.614261,1202,13,1201.887,13.669,1,...,5,0,8,15,79.04,0.0,504,2.89,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
3,4,185.767323,11.059842,185.768468,11.060545,3749,20,3748.309,18.871,1,...,5,0,8,15,79.04,0.0,510,0.26,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
4,5,185.769992,11.408419,185.760949,11.404836,3401,15,3404.483,23.889,1,...,5,0,8,15,79.04,0.0,514,3.07,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
5,6,185.760832,11.405319,185.760956,11.404891,3404,24,3404.428,23.881,1,...,5,0,8,15,79.04,0.0,514,6.30,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
6,7,185.745356,14.457349,185.744486,14.457049,362,17,362.291,17.844,1,...,5,0,8,15,79.04,0.0,514,3.07,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
7,8,185.768394,11.199048,185.768526,11.199208,3610,18,3609.842,17.869,1,...,5,0,8,15,79.04,0.0,515,2.77,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
8,9,185.757436,12.590437,185.756962,12.590814,2222,19,2221.620,19.460,1,...,5,0,8,15,79.04,0.0,514,3.21,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
9,10,185.738710,14.513507,185.738504,14.513499,306,23,306.005,23.199,1,...,5,0,8,15,79.04,0.0,514,3.87,0,/Users/Lyy/Desktop/SummerY1/Yerkes/ca2183/ca21...
